# Geospatial Data in Python with GeoPandas 

A [D-Lab](https://dlab.berkeley.edu) Workshop, Spring 2019

---




# Introduction



The goal of this notebook is to give you a **tip of the iceberg introduction** to working with geospatial data in Python using the **geopandas** package.  Most of the sample data and use cases are related to a UC Berkeley research a project that I have been working on called [The Louisiana Slave Conspiracies](https://dlab.berkeley.edu/landing-page/louisiana-slave-conspiracies). This project explores several slave conspiracies that occured in colonial Louisiana during the late 1700s and early 1800s. Since very little data exist for this time period, we begin with an exploration of US Census data from the early 1800s, shortly after the Louisiana Purchase made the Louisiana and Orleans Territories part of the United States.

> ### Assumptions

> This tutorial assumptions you have basic working knowledge of Python and of geospatial data.   If you need a geospatial refresher, we can start with this a **very** [Brief Introduction to Geospatial Data](https://docs.google.com/presentation/d/1d9GNcLDsnLxfLmrNRNZE976sHN5qNfkU9Rl2gabUsWc/edit?usp=sharing).

 
 
## GeoPandas and related Geospatial Packages

[GeoPandas](http://geopandas.org/) is a relatively new package that makes it easier to work with geospatial data in Python. In the last few years it has grown more powerful and stable. This really is great because previously it was quite complex to work with geospatial data in Python.  GeoPandas is now the go to package for working with geospatial data. 

`GeoPandas` provides convenient, unified access to the functionality of the [pandas](https://pandas.pydata.org/) package , extending it with the geospatial processing capabilities provided by a number of lower level spatial data packages including [shapely](https://pypi.python.org/pypi/Shapely) for geometry processing, [fiona](https://pypi.python.org/pypi/Fiona) and [GDAL/Ogr](https://gdal.org) for spatial data file IO and[ pyproj](https://github.com/jswhit/pyproj) and [PROJ.4](https://github.com/OSGeo/proj.4/wiki) for map projections and coordinate systems.  


We will also use a few other optional geospatial libraries that are  commonly used with geopandas, including:

- **rtree** for spatial indexing to improve performance
- **geopy** for geocoding and for geodesic distance calculations
- **pysal** for spatial analysis functions such as data classification methods.
- **descartes** for ploting Shapelygeometric objects with Matplotlib.


Finally, we will use a number of standard Python libraries including pandas, numpy, and matplotlib.


# Setup



Installing Geopandas can be a bit complex due to the libraries that it depends on.  See the [Geopandas documentation ](http://geopandas.org/install.html) for help with this process - read it carefully as that will save you many headaches!

We will use the [Google Colaboratory](https://colab.research.google.com/notebooks/welcome.ipynb) Jupyter notebook environment for this workshop so that we will all have the same working enviroment.

## About Google Colab

Google Colab is a freemium (*i.e., extra stuff costs $$*) Jupyter notebook environment that requires no setup and runs entirely in the cloud.

- A google account is required!

From the browser you can write and execute Python code and save and share your notebooks.

You can also install libraries that are not readily available and import local or remote data.

- However, the libraries you install and data you import are only available to you temporarily in the Colab environment.

### Why we like Colab

- It's free for our needs

- It's fast

- It removes alot of local package install problems so we can get right to work.

- It ensures that all workshop participants have the same computing environment.

### Learning more

To learn more go to the [Welcome to Google Colab](https://colab.research.google.com/notebooks/welcome.ipynb) site.

# Getting Started

## Open the workshop notebook

To get started, open this notebook and follow along:

- Google Colaboratory Notebook URL: http://bit.ly/geopandas_sp2019

- Once you open the notebook, click **File > Save a copy in Drive** to make a copy of the notebook that you can edit. 

    - This will be saved to a Google Drive Collaboratory folder in your Google account.
    - Close the tab with the version of this notebook that you cannot edit.



## Install Geopandas and dependencies

Google Colaboratory comes with a Juypyter notebook environment with the most common Python packages already installed. To import a library that's not installed by default, you can use **!pip install** or **!apt-get install**.

<br>

To run Geopandas in Google Colab, execute but do not change the code in the following cell. (*The install process we will follow is from [this notebook](https://colab.research.google.com/drive/1tSmJmjD3sTI31Cg1UCIKiE10dBUmWUG7#scrollTo=wHnmdr_QkKec&forceEdit=true&offline=true&sandboxMode=true)*).

>**IMPORTANT** -  if you are installing these Python packages on your local computer see [Geopandas documentation ](http://geopandas.org/install.html) . Do not use the code below as this is for the Google Colaboratory environment.

>> If you have your geopandas enviroment installed locally, you can get the data and notebook for this tutorial are in this github repository: https://github.com/dlab-geo/geopandas_intro



In [0]:
#######################################################
# Code to install geopandas in Google Colaboratory
# You need to run this code each time you run this 
# notebook on Google Colab
# Should take about 1.5 minutes.
#######################################################
%%time 
!apt update
!apt upgrade
!apt install gdal-bin python-gdal python3-gdal 
# Install rtree - Geopandas requirment
!apt install python3-rtree
# Install pysal
!pip install pysal
# Install mapclassify
!pip install mapclassify
# Install Geopandas
!pip install git+git://github.com/geopandas/geopandas.git
# Install descartes - Geopandas requirment
!pip install descartes 

## Import GeoPandas and Related Libraries

Next, import the libraries that we will use.


In [0]:
import pandas as pd
import geopandas as gpd
import mapclassify
import matplotlib.pyplot as plt
from shapely.geometry import Point


## About the Data 

This tutorial uses historical census data for the USA and the Orleans Territory, most of which is now called Louisiana, that were obtained from the `NHGIS`, or *National Historical Geographic Information System* website ([IPUMS NHGIS, University of Minnesota, www.nhgis.org](https://www.nhgis.org)).  A cartographic boundary file for the United States was obtained from the [US Census website](https://www.census.gov/geo/maps-data/data/tiger-cart-boundary.html).


## Fetch the Data with `wget`

The data and related notebooks for this tutorial are in this github repository: https://github.com/dlab-geo/geopandas_intro


However, to keep things simple in the GC environment we will use the command **wget** to fetch the data frthat repo and store it in the `Google Collabortory` environment for the duration of this session.


In [0]:
!wget 'https://raw.githubusercontent.com/dlab-geo/geopandas_intro/master/data/us_states.zip'
!wget 'https://raw.githubusercontent.com/dlab-geo/geopandas_intro/master/data/uscounties_1810.zip'
!wget 'https://raw.githubusercontent.com/dlab-geo/geopandas_intro/master/data/orleans_census_data1810.csv'
!wget 'https://raw.githubusercontent.com/dlab-geo/geopandas_intro/master/data/lsc_points.csv'


### Take a look at the data files

Take a look at the data we will use with the **ls** command (on mac) or the **dir()** command (on windows).

In [0]:
!ls


# GeoPandas - Data Structures

## Reading in a Spatial Data

GeoPandas makes it easy to read in almost any kind of vector data file including the [ESRI Shapefile](https://en.wikipedia.org/wiki/Shapefile) with the [read_file](http://geopandas.org/io.html) command.  You simply put the name of the file in quotes and assign the resulting object to a simple yet informative variable name, here **usa1810**.

You can see from the previous cell that the `usa1810` shapefile is compressed in a zipfile. We can load the zipfile directly in Geopandas.

In [0]:
usa1810 = gpd.read_file("zip://./uscounties_1810.zip")  #US counties in 1810

Now let's take a look at our data.

In [0]:
usa1810.head()

**Important**: Sometimes geopandas cannot read a zipped shapefile due to its content or the way it was created. If this is the case, unzip it and read it in directly.


In [0]:
# Unzip the shapefile
!unzip ./uscounties_1810.zip

# Read shapefile into a geopandas geodataframe object.
usa1810 = gpd.read_file("./uscounties_1810.shp")  #US counties in 1810


Take a minute to review the different syntax used above for reading in a zipped vs unzipped shapefile.

### Discussion - *what is a Shapefile?*


## The Geopandas GeoDataFrame

The `gpd.read_file` command returns a geopandas **GeoDataFrame** object.  We can double-check this with the `type` function.



In [0]:
type(usa1810)

The `GeoDataFrame` is a table-like pandas DataFrame with extra geospatial capabilities. First let's take a look at the GeoDataFrame again using the **head** method.

- *Do you notice anything different about the GeoDataFrame compared to a regular DataFrame?*

In [0]:
usa1810.head()

Because a GeoDataFrame is a pandas DataFrame you can use all the pandas DataFrame methods with it.  Some examples are shown below.


In [0]:
# How many states or territories did the USA have in 1810?

usa1810.STATENAM.nunique()  

In [0]:
# What was the total area in square kilometers of the USA in 1810?

print("The area of the USA in 1810 was %d square meters!" %  usa1810.SHAPE_AREA.sum())

In [0]:
# What were the 5 largest states or territories?
usa1810[['STATENAM','SHAPE_AREA']].groupby(['STATENAM']).sum().sort_values(['SHAPE_AREA'], ascending=False).head(5)

**Suggestion**: If you don't know pandas we recommend you take an online tutorial or D-Lab workshop to get familiar with its methods for data manipulation and analysis.  That will make it easier for you to get the most out of Geopandas.

### Rename columns

Let's rename the county and state name columns to make our work more intuitive moving forward.  We can do this with basic pandas commands.

In [0]:
usa1810.rename(columns={'NHGISNAM' : 'COUNTY', 'STATENAM': 'STATE'}, inplace=True)
usa1810.head()

The special sauce that Geopandas adds to Pandas is the functionality it provides for working with geographic locations. In what column are these found?

## The Geopandas Geometry Column

All Geopandas GeoDataFrames must have one *special* geometry column. This column is named **geometry** by default, but it could be something else. 

When you read in a spatial data file such as a Shapefile to create a new GeoDataFrame the `geometry` column is automatically created. 
 
 
 You can always get the name of your special geometry column:

In [0]:
usa1810.geometry.name



The geometry column is of type **GeoSeries**, taking its name and its base functionality from the pandas series object.   





In [0]:
type(usa1810.geometry)




** Important:** 

You can rename the special geometry column but if you do  you must update the GeoDataFrame's special geometry attribute using **set_geometry**. 

> gdf = gdf.rename(columns={'old_name': 'new_name'}, inplace=True).set_geometry('new_name')

## Challenge

1. Rename the special geometry column **my_geom**  AND reset the value of the special geometry column to this new name using `set_geometry`.
1. Then, run the command `usa1810.geometry.name` again to get the name of the special geometry column
1. Then rename the column back to `geometry` and reset and test the value for the name of the special geometry column.

Note: this is admittedly a bit confusing because the name of the column is `geometry` and it is `the special geometry column`.

In [0]:
# Your code here

## Challenge - Solution

In [0]:
usa1810 = usa1810.rename(columns={'geometry': 'my_geom'}).set_geometry('my_geom')
usa1810.head()

In [0]:
usa1810.geometry.name

In [0]:
type(usa1810.geometry)

In [0]:
usa1810.geometry.name

In [0]:
#Reset the special geometry column to geometry
usa1810 = usa1810.rename(columns={'my_geom': 'geometry'}).set_geometry('geometry')
usa1810.geometry.name

In [0]:
usa1810.head()

## Working with the GeoSeries

A GeoPandas GeoSeries supports three basic types of vector geometries, in [Well-Known Text ](https://en.wikipedia.org/wiki/Well-known_text_representation_of_geometry) format:
- Points / MultiPoints
    - POINT( -122 38)

    - MULTIPOINT((-122 38), (-123 39))
    
- Lines / MultiLines
    - LINE (30 10, 10 30, 40 40)
    
    - MULTILINE((10 10, 20 20, 10 40),(40 40, 30 30, 40 20, 30 10))
    
- Polygons / MultiPolygons
    - POLYGON ((35 10, 45 45, 15 40, 10 20, 35 10), (20 30, 35 35, 30 20, 20 30))
    - MULTIPOLYGON (((30 20, 45 40, 10 40, 30 20)), ((15 5, 40 10, 10 20, 5 10, 15 5)))

Notes:
- GeoPandas does not support raster geometries - check out the [Rasterio](https://rasterio.readthedocs.io/en/latest/) package for that.
- A GeoSeries can contain mixed geometry types but it's not a great idea.

</br>
Let's check what type(s) are in our GeoDataFrame.

In [0]:
set(usa1810.geom_type)  # set returns unique values

Geopandas spatial attributes and methods apply to the special geometry column, even if you do not explicitly reference it.

For example, the code in both of the following cells will return the minimum bounding coordinates that contain all geometries in the GeoSeries.

In [0]:
usa1810.total_bounds

In [0]:
usa1810.geometry.total_bounds

However, most Geopandas geometry methods and attributes apply to **each** geometry in the Geoseries rather than **all** in the aggregrate. 

For example, let's use the bounds attribute to see the bounding coordinates of each county in the usa1810 geodataframe.

In [0]:
usa1810.bounds.head()


Let's take a look at GeoPandas geometries a bit more closely.

First, let's subset the GeoDataFrame to select only the row for the county of New York, New York.

In [0]:
ny_gpd = usa1810[usa1810['COUNTY']=='New York']

print(type(ny_gpd))

ny_gpd

Now let's create a GeoSeries from the geometry.

In [0]:
ny_gs = usa1810[usa1810['COUNTY']=='New York'].geometry
# or
ny_gs = ny_gpd.geometry

print(type(ny_gs))

ny_gs

In [0]:
usa1810[usa1810['COUNTY']=='New York'].geometry.squeeze()

Finally, let's get the geometry value itself.  

To extract a single geometry value from a GeoSeries *with only one row* use the squeeze method.

In [0]:
ny_geom = usa1810[usa1810['COUNTY']=='New York'].geometry.squeeze()
#or
ny_geom = ny_gs.squeeze()

print(type(ny_geom))
ny_geom


When you return a single geometry, the object is plotted. To see the data  in  [well-known text](https://en.wikipedia.org/wiki/Well-known_text_representation_of_geometry) format, or WKT,  use the `print` function.

In [0]:
print(ny_geom)


Squeeze doesn't work on a GeoSeries with more than one element:


In [0]:
nystate_geom = usa1810[usa1810['STATE']=='New York'].geometry

type(nystate_geom)

nystate_geom.head()

You would need to extract each geometry by row index.

In [0]:
nystate_geom[414]

We will look at methods for plotting Geopandas geometries in the next section.

## Summary

GeoPandas has attributes and methods for **GeoDataFrames**, **GeoSeries** and **geometry** objects.

The geometry column returns a GeoSeries even if only one row is requested. But many operations in GeoPandas only work on the geometry value and not the GeoSeries object.

As you work with GeoPandas and read through the [online documentation](http://geopandas.org) keep in mind which type of object you are working with and what type is required as input to a method or returned by a specific method or attribute.




# Mapping GeoDataFrames


One of the first things to do with geographic data once you read it into GeoPandas is visualize it.

We can use the GeoPandas **plot** method to display the data in a GeoDataFrame or GeoSeries. This uses `matplotlib` and the matplotlib `pyplot` module under the hood.

In [0]:
# Plot a GeoDataFrame
usa1810.plot()  # it's really that simple!

We can also plot a subset of the geodataframe.

In [0]:
# Plot all the 1810 counties in New York state
usa1810[usa1810['STATE']=='New York'].plot()

And we can plot a geoseries with plot()

In [0]:
# plot the geometry geoseries
usa1810[usa1810['STATE']=='New York'].geometry.plot()

Pretty cool to be able to make a map with a single command. However, there is always room for improvement. You can find out more about the plotting options for basic maps in the geopandas documentation and in the [matplotlib](https://matplotlib.org/) documentation.

</br>

For now, let's use some options to make a prettier map. Take a minute to consider what each option does.

In [0]:
usa1810.plot(linewidth=0.5, edgecolor='grey', facecolor='pink',  figsize=(14,10) )
plt.show()

When you have time, take a look at the method documentation for **plot** to see all of the available options.


In [0]:
#gpd.GeoDataFrame.plot?
#gpd.GeoSeries.plot?

## Question

Can you think of why the options for plotting a GeoDataFrame are different from those for a GeoSeries?

## Challenge

Let's take a few minutes to practice some of what we have done so far with a different data set.

- Read the file **us_states.zip** into a geopandas dataframe named **usa**.
- Take a look at the data in this dataframe using `head`.
- Then, make a map of the `usa`, 
    - setting the `figsize` to (14,10)
    - the fill color to green,
    - and the outline color to white

In [0]:
# your code here to load the data from the zip file into a geodataframe


In [0]:
# your code here to plot the geodataframe

## Challenge - solution

In [0]:
usa = gpd.read_file('zip://./us_states.zip')
usa.head()

In [0]:
usa.plot(linewidth=0.25, edgecolor='white', facecolor='green',figsize=(14,10))

## Spatial Subsetting

It's never easy to make a nice map of the entire US. Why is that? 

We can zoom in on the contiguous USA by spatially subsetting the data using the GeoPandas **cx** method.  This method takes the form:
>usa1810.cx[xmin:xmax, ymin:ymax]

>where:
- **xmin** is the minimum X coordinate value
- **xmax** is the maximum X coordinate value 
- **ymin** is the minimum Y coordinate value
- **ymax** is the maximum Y coordinate value 

Since our date use geographic coordinates, X values are decimal degrees `longitude` and Y values are in decimal degrees `latitude`.

Let's give it a try.

In [0]:
usa.cx[-130:-80, 25:45].plot(linewidth=0.25, edgecolor='white', facecolor='green',figsize=(14,10))

## Questions

How did that last map turn out?

What exactly is **cx** doing?  Let's explore it a bit more. 

- Change the minimum Y value to 30 and then 35. Do Texas and Florida get clipped?

Take a second to uncomment the command below and read the documentation for `cx`. Then update the values in the previous code cell to get all states.

In [0]:
#usa.cx?

### Saving a spatial subset

We can make that subset permanent.

In [0]:
# FYI: conus is shorthand for contiguous USA
conus= usa.cx[-130:-50, 20:50].reset_index(drop=True)
conus.head()

In [0]:
# Plot the subset
conus.plot()

In [0]:
conus.STATE.nunique()

## *Any questions?*



---



## Map Overlays



A key strength of geospatial data analysis is the ability to overlay data that are located in the same coordinate space. Let's overlay the USA in 1810 on top of the USA in 2017 to visualize the change. Both of these data sets use the same coordinate reference system -  decimal degrees of latitude and longitude referenced to the **World Geodetic System of 1984**, or **WGS84** coordinate reference system (more about that in a minute).

We will explore two methods for doing this, as shown in the [GeoPandas documentation](http://geopandas.org/mapping.html)

The general process is as follows:

- First identify your base map - the layer to draw first, or at the bottom of the stack of layers.
- Then you add the next layer, referencing the base map as the **ax**. 

In [0]:
# Map the us states with the 1810 states and territories overlayed.
base = conus.plot(color='white', edgecolor='black',  figsize=(14,10))
usa1810.plot(ax=base, color="blue", edgecolor="blue")

We can add even more layers. These will draw in the order that you add them. Consider the following code.

In [0]:
base = usa1810.plot(facecolor="blue", edgecolor="blue",  figsize=(14,10))
conus.plot(ax=base, color='None', edgecolor='black')
conus.centroid.plot(ax=base, color="red")  # Hey - what's happening here?

What's different in the code for the previous two maps?

</br>

We can get even fancier with our maps by using the more **matplotlib** options. To access these you need to import mapplotlib.

In [0]:
# Mapping with advanced matplotlib settings

fig, ax = plt.subplots(1, figsize=(14,10))  # Initialize the plot figure (drawing area) and axes (data area)

ax.set_aspect('equal')   # set the aspect ratio for the x and y axes to be equal. 
                         # This is done automatically in gdf.plot()
    
base = usa1810.plot(ax=ax, color='black', edgecolor='black')  # Set the base map, or bottom map layer

conus.plot(ax=base, color='None', edgecolor="blue")  # draw the data with the base
_ = ax.axis('off') # Don't show the x, y axes and labels in the plot
ax.set_title("USA 1810 and 2017")  # Give the plot a title

plt.show()

## Questions?

Many folks, myself included, scratch their head and copy matplotlib code, amazed and mystified when it works. Gradually it sinks in!

# Coordinate Reference Systems (CRS) 



Did you notice anything funny about the **shape** of the USA as mapped above?  How does it differ from the shape of the US in the map below?
<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/a/a5/Map_of_USA_with_state_names.svg/640px-Map_of_USA_with_state_names.svg.png" width="800px"></img>


#### Why does the shape differ? 

Here's why:

<img src="http://tse3.mm.bing.net/th?id=OIP.lyDmHXX9VdoEOWDQlqppSAHaEy" width="500px"></img>

When we map data encoded with a spheriodal coordinates (longitude & latitude) on a 2D plane like a computer screen we get distortion!

## Map Projections and CRS Transformations

In order to reduce distortion in maps we apply a map projection (math) to transform 3D geographic coordinates to 2D projected map coordinates.
<img src="https://www.e-education.psu.edu/natureofgeoinfo/sites/www.e-education.psu.edu.natureofgeoinfo/files/image/projection.gif"></img>


CRS transformations are often necessary for geopandas geometric operations like area and distance calculations which assume a 2D plane.

## Defining and Transforming a CRS
The process for transforming a CRS is:

1. Make sure a **crs** is defined for the geopandas dataframe by checking the **crs** property. 
2. If it is not set, you can **define** it.
3. Transform the coordinate geometry to a new CRS using the **to_crs** method.
- This returns a new geodataframe with the new coordinate values and CRS.
- You need to know what CRS to use!!

Let's check the CRS of our GeoDataFrames

In [0]:
# Check the CRS of our gdfs
print("The CRS of the conus geodata frame is: " + str(conus.crs))
print("The CRS of the usa1810 geodata frame is: " + str(usa1810.crs))

What's an **epsg:4326**?  That's an [EPSG](http://www.epsg.org/) code for the geographic CRS known as the [World Geodectic System of 1984](https://en.wikipedia.org/wiki/World_Geodetic_System#WGS84), or `WGS84`. This is the most commonly used CRS for latitude and longitude coordinate data and is the default CRS for most mapping software when the data does not have a defined CRS.

We can make our map look better by transforming it to a 2D projected `CRS`. A projected CRS applies a mathematical transformation to the data based on a [map projection](https://en.wikipedia.org/wiki/Map_projection)

Common map projections for USA data and their EPSG codes include:

- **Web Mercator** (epsg:3857)
- **USA Contiguous Albers Equal Area** (epsg:5070)

Let's try these with our geodataframe.

First, let's transform the CRS of the **conus** geodataframe to Web Mercator. The **to_crs** method will return a new geododataframe.

In [0]:
# Transform geographic crs to web mercator - 3857
conus_3857 = conus.to_crs(epsg=3857)
conus_3857.plot()

## Challenge

Now you try it! Transform the **conus** geodataframe to one that uses the **USA Contiguous Albers CRS** (7603).

Then, display the output geodataframe.

In [0]:
# Your code here


## Challenge Solution

In [0]:
# Transform the conus geodataframe to USA Albers (7603)
conus_5070 = conus.to_crs(epsg=5070)
conus_5070.plot()

## Multiplots

Let's plot all the data in all 3 CRS together.

In [0]:
fig, ax = plt.subplots(ncols=3, figsize=(18,4), subplot_kw=dict(aspect='equal'))
# Don't show the coordinate axis
ax[0].axis('off')
ax[1].axis('off')
ax[2].axis('off')
# Show a title
ax[0].set_title('Geographic, unprojected (4326)')
ax[1].set_title('Web Mercator (3857)')
ax[2].set_title('Albers EA (5070)')
# display
conus.plot(ax=ax[0])
conus_3857.plot(ax=ax[1])
conus_5070.plot(ax=ax[2])
plt.show()

You can see we get very different maps of the USA depending on the CRS. 

- Geographic coordinates (EPSG 4326) are often used by default when the source data have this CRS. But it shouldn't be used for maps of large areas and is rarely used for spatial analysis.

- Web Mercator is often used for making maps of large geographic extents between +-60 degrees because it preserves shape. This is the CRS used by most online maps like Google Maps. BUT BEWARE - area distortion increases as you move away from equator and towards the poles.  Don't use this CRS for spatial analysis.

- USA Contiguous Albers is used for the maps and area based analysis for the contiguous USA.  For areas within the USA you should use a CRS that is more customized to a specific state or region.

### Important Notes

1. Data usually need to be in the same CRS in order to be mapped or analyzed together.
2. The units of a CRS are part of the CRS definition. These are typically decimal degrees for geographic (lat/lon) data and meters or feet for projected data.
3. It's not obvious what the best projected CRS is for your map or analysis. You need to review the recent literature (as these things change) and try different ones.  Here is a good starting place, [epsg.io](http://epsg.io/).


> A detailed discussion CRS and map projections is beyond the scope of this notebook. Understanding these, however, is **necessary** for working sucessfully with geospatial data! There are a number of online resources that can be found with a web search to help you get started.  Gaining this understanding takes time so be kind to yourself and ask for help if you need it.

## Any Questions?


---



# Spatial Measurements



Geopandas uses the  [Shapely library](https://shapely.readthedocs.io/en/stable/manual.html) to compute spatial measurements like area and length for individual geometries or all the geometries in a geoseries.  The available measurements depend on the geometry type. For example, we can compute area and perimeter for polygons, length for lines, and distances between points.  Read the GeoPandas and Shapely documentation to get a sense of all the meausurements you can compute.

### Calculating Area

Let's compute the area of a single geometry in a geodataframe.

In [0]:
conus[conus['STATE']=='Utah'].area  

Above **aea** returns a pandas series.  You can use *squeeze* to return just the value when the geoseries only has one element.


In [0]:
conus[conus['STATE']=='Utah'].area.squeeze()

**Question**  - What are the units for the above area value?

It doesn't make sense to compute spatial measurements using geographic coordinates. We will address that in a minute. For now let's focus on the measurement methods.

We can compute the area of all geometries in the geodataframe.

In [0]:
#conus.geometry.area
conus.area

Above, we dynamically calculated area. But we can also append it to the geodataframe.

In [0]:
# Update the dataframe
conus['area'] = conus.area
conus.head(15)

### Calculating Length or Perimeter

Similarly we can calculate the perimeter of one or all state polygons.

In [0]:
# Perimeter of the state of Montana
conus[conus['STATE']=='Utah'].length

# Perimeter of all states in the continental USA
# conus.length

### Calculating Distance
We can also compute the shortest distance between individual geometries.

In [0]:
conus[conus['STATE']=='Washington'].squeeze().geometry.distance(conus[conus['STATE']=='California'].squeeze().geometry)

## Spatial Measurements and CRSs

The output of spatial measurements depend on the CRS and is expressed in the units of the CRS. The Shapely library assumes a two dimensional planar coordinate system and makes no transformation on the data - that is left for the analyst.

> **Don't use geographic coordinates for GeoPandas spatial measurement queries. The results in decimal degrees are meaningless!**

Let's compare the output of the area calculations using the different CRSs.



In [0]:
print("Area of the State of Utah:")
print("- CRS Web Mercator: %.2f square KM" % (conus_3857[conus_3857['STATE']=='Utah'].geometry.area / (1000*1000) ))
print("- CRS Albers: %.2f square KM" % (conus_5070[conus_5070['STATE']=='Utah'].area / (1000 * 1000)) )

**Question**

Which CRS returns the (most) correct value for the area of Utah? Check [Wikipedia](https://en.wikipedia.org/wiki/Utah)...

*Discussion*


## CRSs and Distance Calculations

Let's compute the distance between Washington & California using the Web Mercator and Albers CRSs.

- *Note you need to extract the geometry value using **squeeze** for the following distance calculation*

In [0]:
# Your code here
wm_dist_m = conus_3857[conus_3857['STATE']=='Washington'].squeeze().geometry.distance(conus_3857[conus_3857['STATE']=='California'].squeeze().geometry)
wm_dist_km = wm_dist_m / 1000
print("web mercator dist KM:", wm_dist_km)
#
al_dist_m = conus_5070[conus_5070['STATE']=='Washington'].squeeze().geometry.distance(conus_5070[conus_5070['STATE']=='California'].squeeze().geometry)
al_dist_km = al_dist_m / 1000
print("Albers dist KM:", al_dist_km)


 ## Question
 
 Which of the above CRSs returned the best result?  Let's check it in [Google Maps](http://maps.google.com) to find out.

## Any Questions?


---



# Spatial Relationship Queries



[Spatial relationship queries](https://en.wikipedia.org/wiki/Spatial_relation) consider how two geometries or sets of geometries relate to one another in space. Fortunately, these don't depend on the CRS of the data.

<img src="https://upload.wikimedia.org/wikipedia/commons/5/55/TopologicSpatialRelarions2.png" height="400px"></img>


Let's consider some spatial relationship queries between individual geometries and geopandas geodataframes / geoseries.

To start, let's create  geometry objects that represent Louisiana and the Parish of Pointe Coupee.

In [0]:
# Louisiana today
la_poly = conus[conus['STATE']=='Louisiana'].geometry.squeeze()
la_poly

In [0]:
# The Parish (county) of Pointe Coupee Louisiana in 1810
ptcoupee_poly = usa1810[usa1810['COUNTY']=='Pointe Coupee'].geometry.squeeze()
ptcoupee_poly



In [0]:
# A Point representation of the center point (centroid) of Pointe Coupee Louisiana in 1810
ptcoupee_pt = usa1810[usa1810['COUNTY']=='Pointe Coupee'].geometry.centroid.squeeze()
ptcoupee_pt

In [0]:
# The Orleans Territory in 1810
orleans_poly = usa1810[usa1810['STATE']=='Orleans Territory'].geometry.squeeze()
orleans_poly


Above, we get a geoseries for Orleans Territory instead of a geometry because the rows in usa1810 represent counties not states or territories. 

In [0]:
type(orleans_poly)

In [0]:
# If we make these geometries a geoseries we can quickly plot together
geom_data = gpd.GeoSeries([la_poly, ptcoupee_poly, ptcoupee_pt]).plot(cmap="Reds",figsize=(10,10))
orleans_poly.plot(ax=geom_data, color='None', edgecolor="grey")

Now let's consider the following spatial relationship query:

- Is Pointe Coupee within Orleans Territory?


Here is a list of the GeoPandas methods we can use to test spatial relationships.

*Which one(s) can we use to answer the above query?*

- equals
- contains
- crosses
- disjoint
- intersects
- overlaps
- touches
- within
- covers

In [0]:
# spatial relationship query between two geometry objects
ptcoupee_poly.within(la_poly)


**Question** - How do you think we would  restate that using ** contains**?

In [0]:
# Our code here

We can test the spatial relationship between a geometry object and all geometries in a GeoDataFrame.

In [0]:
usa1810.contains(ptcoupee_poly)

Can we restate the previous operation with **within**?

- Uncomment the code below to see....

In [0]:
#ptcoupee_poly.within(usa1810)

Since spatial functions return **True** or **False** we can use them to select only the rows in the GeoDataFrame that meet the condition being tested.

In [0]:
# What state or territory contained Pointe Coupee in 1810?
usa1810[usa1810.contains(ptcoupee_poly)]

In [0]:
# What US state contains pt coupee today?


## Challenge

What US State is Pointe Coupee within today?


In [0]:
#Your code here

## Challenge - solution

In [0]:
# Your code here
conus[conus.contains(ptcoupee_poly)]

**Discussion**


##  Intersects - the most general spatial relationship query

The most useful, fastest and most general purpose spatial relationship query is **intersects**. You don't need to worry about selecting the correct spatial relationship predicate for your query or differences due to the resolution and alignment of your geometries.

</br>

Below we use *intersects* to see what states border Louisiana.


In [0]:

conus[conus.intersects(la_poly)]

Intersects is not a relative operator like contains or withins. You can compare two geometries in any order and get the same result.

In [0]:
print(ptcoupee_poly.intersects(la_poly))

print(la_poly.intersects(ptcoupee_poly))

## Any Questions?



---



# Spatial Data Processing



Spatial relationship queries return `True` or `False` when comparing geometries based on a spatial relationship predicate. 

Geometric processing operations, on the other hand, construct new geometries from one or more input geometries. We saw this earlier with the **dissolve** operation.

These transformations, which are also called **geoprocessing**, make up the bulk of spatial preprocessing operations - the work you do to prepare your data for analysis!





## Common Types of Geoprocessing operations

Below is a list of some common types of geoprocessing operations.

- Coordinate system transformations
- Dimensionality transformations (points to polygons, polygons to points or lines, lines to polygons or points)
- Geometric Aggregations (simplify, dissolve / group by operations)
- Spatial overlay operations

An in-depth review of all of the types of and methods for geoprocessing is beyond the scope of this workshop.   A good way to get an overview is to work through the different sections of the Geopandas documentation.

Instead, we will work through a few of these as we explore our historical Louisiana data.

## Question

Can you identify the types of geoprocesing operations we have done so far?

## Geometric Aggregrations

We often receive data that has more detail than we need. For example, we might have population by county when we want it by state. In pandas you can use a `groupby` operation to aggregate the data values.  

With geospatial data we often obtain data that has more geometric detail than we need. We can use the geopandas **dissolve** method to aggregate geometric data that share a common value.

For example, let's create a single geometry that is the outline of the continental US by dissolving the state geometry in the `conus` geodataframe. 
Start by taking a look at the geodataframe again.


In [0]:
conus.head()

We need a column value to indicate how to aggregate the geometry. We can create this column and populate it if it doesn't exist. 


In [0]:
# create a column with a single value that we can dissolve on
conus['country'] = 'usa' 
conus.head()

Now we can use the **dissolve** method to merge all the states that have the value "usa" in the country column.

Note, **dissolve** won't know what to do with the values specific to each state - like STATE (name), GEOID, ABBREV, so we remove those columns first.

In [0]:
# Select on the columns we weant to keep in dissovled geodataframe
conus_outline = conus[["country","geometry"]]  

# dissolve the interior polygons
conus_outline = conus_outline.dissolve(by='country', as_index=False)

# take a look
conus_outline.head()



Finally, **plot** the dissolved geodataframe.

In [0]:
conus_outline.plot()

## Challenge

1. Use **dissolve** to create the new GeoDataFrame **usa1810_outline**
    - *Note: you do not need to add a new column - what column should you use to dissolve by?*


In [0]:
usa1810.head(2)

In [0]:
# your code here


### Challenge - solution

In [0]:

# Select on the columns we weant to keep in dissovled geodataframe
usa1810_outline = usa1810[["DECADE","geometry"]]  

# dissolve the interior polygons
usa1810_outline = usa1810_outline.dissolve(by='DECADE', as_index=False)

# take a look
#usa1810_outline.head()
usa1810_outline.plot()

## Challenge 

Display the **usa1810_outline** on top of the **conus_outline**

- *HINT: You can do this quickly by copying related code from a previous section and changing the geodataframe names*

In [0]:
# Your code here

### Challenge - solution

In [0]:
# your code here
base = conus_outline.plot(color='white', edgecolor='black',  figsize=(14,10))
usa1810_outline.plot(ax=base)

## Spatial Overlay
The [GeoPandas **overlay** operation](http://geopandas.org/set_operations.html) allows you to compare two geodataframes and return the set intersection, union, difference between them. 

As an example, let's consider how we can use **overlay** to create a new geodataframe that represents the areas that were added to the continental United States after 1810.

*Note: these operations can be quite computationally intensive on complex geometries!*

In [0]:
# What states were added after Louisiana Purchase
usa_post1810 = gpd.overlay(conus, usa1810_outline, how='difference')

In [0]:
usa_post1810.plot(color="red", figsize=(14,10))

You can see in the map above that the spatial difference operation returned sliver geometries where the two datasets didn't completely align. This is extremely common.

## Question

How could you use the data in the output GeoDataFrame to remove the `sliver` polygons?

In [0]:
usa_post1810.head()




---



## Spatial Joins

Spatial join operations are used to join attribute data from one spatial object to another **based on spatial location**.  In other words, we can transfer attributes from one GeoDataFrame to another GeoDataFrame for objects that are colocated in space.

</br>
In `geopandas` this is done with the **sjoin** operator.  Spatial joins borrow syntax from non-spatial attribute table joins.

Take a look at the documentation for **sjoin**.

Let's explore spatial joins by:

- reading in a csv file of Louisiana slave conspiracy locations into a new GeoPandas GeoDataFrame.
- using a spatial join to add the name of the parish in which the conspiracy took place to the slave conspiracy GeoDataFrame.

In [0]:
# Read in a csv file that has lat/lon values
lsc_locs = gpd.read_file("./lsc_points.csv")
 
#take a look at the data
lsc_locs

Because we have read in a CSV file rather than a spatial data file, we have a GeoDataFrame with no data in the special **geometry** column.   However, we do have coordinate data in the latitude and longitude columns. We can use some code to convert those coordinates to Point geometry.

In [0]:
# populate geometry column from lat/lon data
lsc_locs['geometry'] = lsc_locs.apply(lambda row: Point(float(row['longitude']), float(row['latitude'])), axis = 1)



The code above uses a `lamda` function to take each row in the GeoDataFrame and constructs a Point geometry object from the longitude and latitude data and stores it in the geometry column.
</br>

Let's  take a look at the GeoDataFrame now.


In [0]:
lsc_locs

In [0]:
# And plot the locations:
base = orleans_poly.plot()
lsc_locs.plot(ax=base, color='red')

## SJOIN

We are now ready to use **sjoin** to add the parish (county) for each conspiracy.

In [0]:
lsc_with_parish = gpd.sjoin(lsc_locs, usa1810)



*What went wrong?*

### CRSs must be the same for spatial joins.

First take a look at the crs of the lsc_locs data.

In [0]:
lsc_locs.crs

*Why isn't the CRS defined?*  Because we read the data in from a CSV file which is not a spatial data file format. Thus it does not contain CRS information.

## Defining a CRS

If you know the CRS of your data you can define it by referencing its **epsg** code. 

*Note: defining a CRS is not a transformation!*

In [0]:
lsc_locs.crs = {'init': 'epsg:4326'}

# If we know two geodataframes have the same CRS
# We can also define the CRS like this:

#lsc_locs.crs = usa1810.crs 

Now try that spatial join again!

In [0]:
lsc_with_parish = gpd.sjoin(lsc_locs, usa1810)


Take a look at the results...

In [0]:
lsc_with_parish.head()


Now you can answer our question:

- In what parishes (counties) did the three Louisiana Slave Conspiracies take place?


## Attribute Joins

Attribute joins combine data from different tables based on a column with shared values.  Although these are not spatial they are widely used in geospatial analysis and in all data analysis.  We use the **merge** command for geopandas attribute joins.

<br>
Let's use an attribute join to join some census data for Orleans Territory to a subset of the usa1810 data.

First, read in the CSV file to a Pandas DataFrame named **orleans_census1810**



In [0]:
orleans_census1810 = pd.read_csv('./orleans_census_data1810.csv')
orleans_census1810.head()

Then, subset `usa1810` to a new GeoDataFrame keeping only the data where the STATE is Orleans Territory - name this gdf  **orleans**.


In [0]:
orleans = usa1810[usa1810['STATE'] == 'Orleans Territory']

orleans.plot()



First, let's take a look at the values in the **orleans** GeoDataFrame.  Compare it to the **orleans_census1810** data frame.

- What column should we use for the join?

In [0]:
orleans.head()

Join the attribute data in **orleans_census1810** to the **orleans** GeoDataFrame using the **merge** command.

In [0]:

orleans_popdata = orleans.merge(orleans_census1810, on='GISJOIN')
orleans_popdata.head()

You can see that we now have a number of population related attributes in the geodataframe.

What happened to the columns that were in both dataframes?

## Questions?



---



# Data Driven Mapping



Data driven mapping refers to the process of creating thematic maps by using data values to determine the symbology of mapped features - including their color, shape, size.  This is in contrast to setting the same symbology for all features as we have done above.

### Mapping categorical data
We can symbolize the color of our features by a categorical data value.

In [0]:
conus.plot(column="STATE", edgecolor="white")


### Mapping quantitative data

We can also color areas by quantitave data values. 

</br>

Let's map the parishes in Orleans Territory by the number of non-white slaves. These values are in the column `nwslave_pop`.


In [0]:
orleans_popdata.plot(column='nwslave_pop', cmap='Reds', edgecolor='grey', legend=True, figsize=(8,6))
plt.show()

### cmaps - colormaps

Note the use of the **cmap** option to set the matplotlib color palette for mapping the data.  Take a look at the [documumentation](https://matplotlib.org/users/colormaps.html) for these colormaps and rerun the previous code with a different value for **cmap**.   I strongly recommend that you read this documentation to improve your use of colormaps to effectively map data values. 

### Discussion

Above, the plot option **column=** tells the plot command to use the values in the **nwslave_pop** column to determine the geometry colors based on the colormap specified by the **cmap** option. You can see the list of available [color maps here](https://matplotlib.org/users/colormaps.html). The full range of values in the `nwslave_pop` column is being scaled to the color palette called **Reds**.  This is called an `unclassified` or `classless` map. This map is a good first effort as it imposses no grouping on the data, thus making it easier to spot trends and outliers. But it is harder to interpret the data values within an area. 


### Graduated Color Maps

A more common practice is to use a **classification scheme** to bin data values into 4-7 classes and map those classes to a color palette.  This type of map is called a **graduated color map**.

</br>

Let's try that below with **quantile** classification which is the most commonly used scheme when mapping data.

In [0]:

orleans_popdata.plot(column='nwslave_pop', cmap='Reds', edgecolor='black', 
                     legend=True, figsize=(8,6), scheme='quantiles')
plt.show()

Wow that gives a very different looking map!

## Challenge

In the code cell below recreate the above map with the classification schemes **equal_interval** and **fisher_jenks** to see how the look of the map changes.


In [0]:
# Your code here

## Choropleth Maps

The maps we just made are called `choropleth maps`. A [choropleth map](https://en.wikipedia.org/wiki/Choropleth_map) is a data map that colors areas by data values.  This are the most common type of data map. It is also sometimes called a **heatmap**.

<br>
**Important**, when the areas being mapped vary in size it is not considered good cartographic practice to map **counts**.  Why do you think this is so?

Instead, choropleth maps typically symbolize ares by area weighted densites, ratios or rates that can compared across the different sized areas.

<br>

Let's map the ratio of non-white slaves (nwslave_pop) to free whites (white_pop).

In [0]:
# Create a new column that is the ratio of non-white slaves (nwslave_pop) to free whites (white_pop)
orleans_popdata['slave2white_ratio'] = orleans_popdata['nwslave_pop'] / orleans_popdata['white_pop']

# Map the ratio
orleans_popdata.plot(column='slave2white_ratio', cmap='Reds', edgecolor='black', 
                     legend=True, figsize=(8,6), scheme='quantiles')

plt.show()

In [0]:
orleans_popdata.head()

Let's redo the above map by adding labels and a few more niceties.

We will also use **fisher_jenks** classification to minimize within bin variance and maximize between bin variance. This creates groupings that better reflect the data.

In [0]:
fig, ax = plt.subplots(1, figsize=(12,12))

orleans_popdata.plot(ax=ax, column='slave2white_ratio', cmap='OrRd', edgecolor='black', legend=True, scheme='fisher_jenks')

for polygon, name in zip(orleans_popdata.geometry, orleans_popdata.COUNTY_x):
    ax.annotate(xy=(polygon.centroid.x, polygon.centroid.y), s=name)

_ = ax.axis('off')

ax.set_title("Ratio of Non-White Slaves to Free Whites, Orleans Territory, 1810")
plt.show()

Needless to say, labels are a bit tricky, regardless of the software you use to make a map!


## Question

In 1791 and 1795 two slave revolts were planned in the same parish in Orleans Territory. Although these plots involved different people and had different orgins, both were discovered and thwarted, leading to the trial and execution or emprisonment of many enslaved persons. Soon thereafter, the [German Coast Uprising of 1811](https://en.wikipedia.org/wiki/1811_German_Coast_uprising), which was the largest slave revolt in US history, occured in a different Orleans parish. 

- *Does the map symbology indicate the two parishes in which these three events occured?*

As a check, we can add the **lsc_locs** points to the map above.

In [0]:
fig, ax = plt.subplots(1, figsize=(12,12))

orleans_popdata.plot(ax=ax, column='slave2white_ratio', cmap='OrRd', edgecolor='black', legend=True, scheme='fisher_jenks')

lsc_locs.plot(ax=ax, color='yellow', edgecolor="black", linewidth=3, markersize=100)

for polygon, name in zip(orleans_popdata.geometry, orleans_popdata.COUNTY_x):
    ax.annotate(xy=(polygon.centroid.x, polygon.centroid.y), s=name)

_ = ax.axis('off')

ax.set_title("Ratio of Non-White Slaves to Free Whites, Orleans Territory, 1810")
plt.show()

###  Any Questions?



---



# Interactive Mapping with Folium

[Folium] is the most commonly used Python library for creating interactive maps.  See the online documentation for details. 

Below are a few examples for you to consider.

First load the library. Install it if necessary.

In [0]:
#!pip install folium
import folium

Create a simple interactive map centered on Pointe Coupee and with an intial zoom level of 10.

In [0]:
m = folium.Map(location=[ptcoupee_pt.y, ptcoupee_pt.x], tiles='Stamen Toner',
    zoom_start=10)
m

### Create a choropleth map in folium.

In [0]:
m = folium.Map(location=[ptcoupee_pt.y, ptcoupee_pt.x], tiles='Stamen Toner',
    zoom_start=8)

m.choropleth(orleans_popdata, data=orleans_popdata, key_on='feature.properties.GISJOIN',
             columns=['GISJOIN', 'nwslave_pop'], fill_color='YlOrBr')
m

### Point markers with Popups

First, need to make sure the coordinates are numeric!

In [0]:
lsc_locs['latitude'] = pd.to_numeric(lsc_locs["latitude"])
lsc_locs['longitude'] = pd.to_numeric(lsc_locs["longitude"])

In [0]:
m = folium.Map(location=[ptcoupee_pt.y, ptcoupee_pt.x], tiles='Stamen Toner',
    zoom_start=10)

for i in lsc_locs.index:
  folium.CircleMarker(
    location=[lsc_locs.latitude[i], lsc_locs.longitude[i]],
    radius= 10,
    popup= lsc_locs.name[i],
    color='red',
    fill=True,
    fill_color='red'
).add_to(m)


m

### Overlay points on our choropleth map.

In [0]:
m = folium.Map(location=[ptcoupee_pt.y, ptcoupee_pt.x], tiles='Stamen Toner',
    zoom_start=8)

m.choropleth(orleans_popdata, data=orleans_popdata, key_on='feature.properties.GISJOIN',
             columns=['GISJOIN', 'nwslave_pop'], fill_color='YlOrBr')

for i in lsc_locs.index:
  folium.CircleMarker(
    location=[lsc_locs.latitude[i], lsc_locs.longitude[i]],
    radius= 10,
    popup= lsc_locs.name[i],
    color='red',
    fill=True,
    fill_color='red'
).add_to(m)
  

m

## Any Questions?

# Going Further



### Start with the Package Documentation

- [GeoPandas Documentation](http://geopandas.org/)
- [Shapely Documentation(https://shapely.readthedocs.io/en/stable/)]

### I highly recommend this SciPy 2018 workshop on geopandas.

- The notebooks are [here](https://github.com/geopandas/scipy2018-geospatial-data).
- And be sure to watch the [youtube video](https://www.youtube.com/watch?v=kJXUUO5M4ok). 

### More Geopandas Practice

- Try this [Geopandas tutorial](https://www.datacamp.com/community/tutorials/geospatial-data-python) on plotting the path of Hurricane Florence.



### Interactive mapping 

- The [mplleaflet](https://github.com/jwass/mplleaflet) and [folium](https://github.com/python-visualization/folium) packages are very popular for creating interactive web maps in python notebooks. Check out the online documentation and do a web search for an online tutorial to get started.


# Thank you!


---

Last updated 05/08/2019 by Patty Frontiera (pattyf [at] berkeley [dot] edu)